In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
from scores import scores
import time
import warnings
warnings.filterwarnings("ignore")

### 4.) Import Data

In [3]:
%run ../notebooks/utils.ipynb

In [4]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")
#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")
train_complete_Xy_original_df = train_complete_Xy_original_df[train_complete_Xy_original_df.trustLevel <=2]

In [5]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [6]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_scaled_df = transformer.apply_scaler(transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud'])))
train_complete_y_df = train_complete_Xy_original_df.copy().fraud

test_final_X_df = transformer.add_features(test_final_X_df)


display(train_complete_X_scaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
5,-0.893396,-0.263058,-1.274137,1.593561,0.040410,-0.264430,-0.108359,-0.208066,-0.221215,1.208549
7,-0.235897,1.198479,-0.873056,-1.564100,0.980740,0.905411,-0.235251,-0.207616,-0.570397,-0.653125
9,-0.235897,-0.347922,-0.246945,1.306501,-0.899920,0.905411,-0.092203,-0.154386,-0.264716,1.324903
15,-0.893396,-0.074473,-0.542532,-0.702920,-1.213363,1.490331,-0.233279,-0.178982,-0.157728,-1.118544
23,-0.235897,-1.479434,-0.780578,-0.128800,0.353853,-0.264430,0.627285,0.032768,-0.398451,0.975839


5     1
7     0
9     0
15    0
23    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [7]:
score = scores.Scores()

In [8]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [9]:
params = {
    "tol": uniform(1e-5, 1e-1),  # default 100
    "C" :  uniform(0.0, 80.0),
    "shrinking" : [True, False]
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [12]:
default_svm = SVC(kernel="linear", probability=True, cache_size=8000,  verbose=0, random_state=42)
default_svm.get_params().keys()

dict_keys(['shrinking', 'class_weight', 'kernel', 'random_state', 'decision_function_shape', 'C', 'verbose', 'cache_size', 'gamma', 'max_iter', 'coef0', 'degree', 'probability', 'tol'])

In [10]:
train_complete_X_scaled_df.describe()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
count,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000
mean,-0.557384,0.004086,0.007988,0.027202,-0.014062,0.045690,-0.045023,-0.024500,0.032416,-0.029540
std,0.328912,1.009082,1.003128,0.983496,0.998876,0.984601,0.608506,0.924051,1.026643,1.012251
min,-0.893396,-1.711394,-1.651585,-1.564100,-1.526806,-1.434270,-0.262779,-0.226304,-0.570397,-1.700317
25%,-0.893396,-0.861817,-0.834695,-0.702920,-0.899920,-0.849350,-0.225114,-0.194434,-0.432840,-0.885835
50%,-0.235897,-0.017897,-0.087334,-0.128800,0.040410,0.320491,-0.186890,-0.159952,-0.260895,-0.071352
75%,-0.235897,0.874112,0.912800,1.019441,0.980740,0.905411,-0.109766,-0.082631,0.056543,0.859485
max,-0.235897,1.735947,1.764969,1.593561,1.607626,1.490331,7.224870,21.412191,7.682984,1.673967


In [13]:
search = RandomizedSearchCV(default_svm, scoring=scoring, param_distributions=params, random_state=42, n_iter=10000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')
search.fit(train_complete_X_scaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 287 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 787 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 1487 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 2387 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 3487 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 4787 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 6287 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 7987 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 9887 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 11987 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 14287 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 16787 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 19487 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 22387 tasks 

In [14]:
train_complete_Xy_scaled_df = train_complete_X_scaled_df.copy()
train_complete_Xy_scaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_scaled_df = train_X_wo_knn_scaled_df.copy()
train_Xy_wo_knn_scaled_df['fraud'] = train_y_wo_knn_df

val_Xy_scaled = val_X_scaled_df.copy()
val_Xy_scaled['fraud'] = val_y_df.copy()

In [15]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_scaled_df, df_val=val_Xy_scaled)

Results Fix Split: 
DMC Score: 70  ---  Normalized DMC Score: 0.18617021276595744, 

Results Cross Validation: 
DMC Score: 64.0  ---  Normalized DMC Score: 0.17025531914893616 


In [16]:
dmc = np.mean(search.cv_results_['mean_test_DMC'])
ap = np.mean(search.cv_results_['mean_test_AP'])
precision = np.mean(search.cv_results_['mean_test_Precision'])
recall = np.mean(search.cv_results_['mean_test_Recall'])

In [17]:
print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

DMC: 83.36904565537556 --- Average Precision: 0.9752314186688406 --- Precision: 0.9176624287686971 --- Recall: 0.9565885964282618


In [20]:
search.best_estimator_

SVC(C=5.675273359994213, cache_size=8000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=42,
  shrinking=False, tol=0.07958926694361011, verbose=0)

In [21]:
search.best_params_

{'C': 5.675273359994213, 'shrinking': False, 'tol': 0.07958926694361011}